In [ ]:
!pip install datasets evaluate transformers[sentencepiece] -q
!pip install accelerate -q
!apt install git-lfs -q
!pip install wandb

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from huggingface_hub import login

login(token=HUGGINGFACE_API_KEY)

In [ ]:
import wandb
wandb.login(key=WANDB_API_KEY)

In [ ]:
from datasets import load_dataset, DatasetDict, concatenate_datasets

# Load datasets
raw_datasets1 = load_dataset("squad") 

# Merge train datasets
merged_train = concatenate_datasets([raw_datasets1["train"], raw_datasets2["train"]])

# Shuffle and split into train and validation
train_dev_datasets = merged_train.shuffle(seed=42).train_test_split(train_size=0.9, seed=42)

# Rename test to validation
train_dev_datasets["validation"] = train_dev_datasets.pop("test")

# Print dataset info
print(train_dev_datasets)


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoTokenizer

trained_checkpoint = "Yunika/muril-large-squad"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)

In [ ]:
max_length = 512
stride = 256

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")

    answers = examples["answers"]

    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]

        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])

        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])): # loop through featues
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
train_dataset = train_dev_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_dev_datasets["train"].column_names,
)
validation_dataset = train_dev_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=train_dev_datasets["validation"].column_names,
)
len(train_dev_datasets["train"]), len(train_dataset), len(train_dev_datasets['validation']), len(validation_dataset)


In [ ]:
from tqdm.auto import tqdm
import numpy as np
import collections
import evaluate

metric = evaluate.load("squad")
n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    
                    try:
                        answer = {
                            "text": context[offsets[start_index][0] : offsets[end_index][1]],
                            "logit_score": start_logit[start_index] + end_logit[end_index],
                        }
                        answers.append(answer)
                    except:
#                         print(context)
                        continue
                          
#                         print(start_index, offsets[start_index])
#                         print(end_index, offsets[end_index])

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                # {"id": example_id, "prediction_text": best_answer["text"]}
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            # predicted_answers.append({"id": example_id, "prediction_text": ""})
            predicted_answers.append({"id": str(example_id), "prediction_text": ""})

    # theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    theoretical_answers = [{"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples]
    print(predicted_answers[0])
    print(theoretical_answers[0])
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)
    # return {'Training Loss': 0.5, 'Validation Loss': 0.6}

In [ ]:
from transformers import AutoModelForQuestionAnswering


model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "Yunika/muril-large-squad-en-np-hi",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, train_dev_datasets["validation"])

In [ ]:
trainer.push_to_hub(commit_message="Translated Training complete")